In [21]:
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta

In [5]:
with open('../data/BusData.txt', 'r') as f:
    text = f.readlines()
    
l = []
for item in text:
    if item.startswith('H.') or item.startswith('U.'):
        l.append(item.strip('\n').split())

In [6]:
col = ['From', 'Week_day', 'Dept_time', 'Time_take', 'Arr_time']
df = pd.DataFrame(l, columns=col)
df.head()

,From,Week_day,Dept_time,Time_take,Arr_time
0,U.,Mon,12h05,13m55,None
1,H.,Tue,08h36,14m19,None
2,U.,Tue,15h05,14m14,None
3,H.,Wed,06h41,14m48,None
4,U.,Wed,10h12,13m04,None


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   From       404 non-null    object
 1   Week_day   404 non-null    object
 2   Dept_time  404 non-null    object
 3   Time_take  365 non-null    object
 4   Arr_time   303 non-null    object
dtypes: object(5)
memory usage: 15.9+ KB


In [8]:
df.isnull().sum()

From           0
Week_day       0
Dept_time      0
Time_take     39
Arr_time     101
dtype: int64

In [9]:
df.fillna('.', inplace=True)

In [10]:
df.Week_day.unique()

array(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sun', 'Sat', 'Thu.', 'Fri.',
       'Mon.', 'Tue.', 'Wed.', 'Sun.', 'Sat.'], dtype=object)

In [11]:
for i in col:
    df[i] = df[i].apply(lambda x: x.strip('. '))

In [12]:
df.Week_day.unique()

array(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sun', 'Sat'], dtype=object)

In [13]:
df

,From,Week_day,Dept_time,Time_take,Arr_time
0,U,Mon,12h05,13m55,
1,H,Tue,08h36,14m19,
2,U,Tue,15h05,14m14,
3,H,Wed,06h41,14m48,
4,U,Wed,10h12,13m04,
...,...,...,...,...,...
399,H,Mon,09h48,,
400,H,Tue,12h26,,
401,H,Thu,06h40,,
402,H,Fri,09h30,,


In [14]:
df[df.Dept_time.str.contains('m')]

,From,Week_day,Dept_time,Time_take,Arr_time
134,H,Mon,06h42.15m55,,
276,U,Thu,16m33,17m32,


In [16]:
df.iloc[134].Time_take = '15m55'
df.iloc[134].Dept_time = '06h42'
df.iloc[276].Dept_time = '16h33'
df.iloc[[134, 276]]

,From,Week_day,Dept_time,Time_take,Arr_time
134,H,Mon,06h42,15m55,
276,U,Thu,16h33,17m32,


In [18]:
df.head()

,From,Week_day,Dept_time,Time_take,Arr_time
0,U,Mon,12h05,13m55,
1,H,Tue,08h36,14m19,
2,U,Tue,15h05,14m14,
3,H,Wed,06h41,14m48,
4,U,Wed,10h12,13m04,


In [22]:
# chuyển định dạng str cột dept_time & arr_time về date time (tạo cột mới hoặc sửa trực tiếp trên cột cũ)
def time_convert(x):
    if x != '':
        return datetime.strptime(x, '%Hh%M')
df['Dept'] = df.Dept_time.apply(time_convert)

In [23]:
df['Arr'] = df.Arr_time.apply(time_convert)

In [24]:
def time_delta(x):
    if x != '':
        return timedelta(seconds=60*int(x[:2]) + int(x[-2:]))
df['Time_gap'] = df.Time_take.apply(time_delta)

In [25]:
df.head()

,From,Week_day,Dept_time,Time_take,Arr_time,Dept,Arr,Time_gap
0,U,Mon,12h05,13m55,,1900-01-01 12:05:00,NaT,0 days 00:13:55
1,H,Tue,08h36,14m19,,1900-01-01 08:36:00,NaT,0 days 00:14:19
2,U,Tue,15h05,14m14,,1900-01-01 15:05:00,NaT,0 days 00:14:14
3,H,Wed,06h41,14m48,,1900-01-01 06:41:00,NaT,0 days 00:14:48
4,U,Wed,10h12,13m04,,1900-01-01 10:12:00,NaT,0 days 00:13:04


In [26]:
# thời gian di chuyển trung bình (chưa fill n/a)
df.Time_gap[df.Time_gap.notnull()].mean()

Timedelta('0 days 00:14:41.989795918')

In [27]:
# fill n/a thời gian di chuyển = dept - arr
df_dep_arr = df[(df.Dept.notnull()) & (df.Arr.notnull())]
df_dep_arr['Gap'] = df_dep_arr.Arr - df_dep_arr.Dept
df.Time_gap[(df.Dept.notnull()) & (df.Arr.notnull())] = df_dep_arr.Gap

<ipython-input-27-0b920c61d384>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dep_arr['Gap'] = df_dep_arr.Arr - df_dep_arr.Dept
<ipython-input-27-0b920c61d384>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Time_gap[(df.Dept.notnull()) & (df.Arr.notnull())] = df_dep_arr.Gap


In [28]:
# fill n/a giờ arr = dept + gap
df_dep_gap = df[(df.Dept.notnull()) & (df.Time_gap.notnull())]
df_dep_gap.Arr = df_dep_gap.Dept + df_dep_gap.Time_gap
df.Arr[(df.Dept.notnull()) & (df.Time_gap.notnull())] = df_dep_gap.Arr

/home/tm/.local/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-28-9405723adb4f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Arr[(df.Dept.notnull()) & (df.Time_gap.notnull())] = df_dep_gap.Arr


In [29]:
# thời gian di chuyển trung bình (đã fill n/a)
df.Time_gap[df.Time_gap.notnull()].mean()

Timedelta('0 days 00:14:46.919642857')

In [30]:
df.isnull().sum()

From           0
Week_day       0
Dept_time      0
Time_take      0
Arr_time       0
Dept          41
Arr          139
Time_gap     180
dtype: int64

In [31]:
# drop n/a
df2 = df.dropna()[['From', 'Week_day', 'Dept', 'Time_gap', 'Arr']]

In [32]:
# tạo cột giờ trong ngày để nhóm lại
df2['Day_time'] = df2.Dept.apply(lambda x: x.time().hour)

# tạo cột gap_in_second để tính thời gian
df2['gap_in_second'] = df2.Time_gap.apply(lambda x: x.seconds)

In [33]:
df2.head()

,From,Week_day,Dept,Time_gap,Arr,Day_time,gap_in_second
0,U,Mon,1900-01-01 12:05:00,0 days 00:13:55,1900-01-01 12:18:55,12,835
1,H,Tue,1900-01-01 08:36:00,0 days 00:14:19,1900-01-01 08:50:19,8,859
2,U,Tue,1900-01-01 15:05:00,0 days 00:14:14,1900-01-01 15:19:14,15,854
3,H,Wed,1900-01-01 06:41:00,0 days 00:14:48,1900-01-01 06:55:48,6,888
4,U,Wed,1900-01-01 10:12:00,0 days 00:13:04,1900-01-01 10:25:04,10,784


In [34]:
# tạo bảng df_H chỉ gồm thời gian đi từ nhà
df_H = df2[df2.From=='H']
df_H.head()

,From,Week_day,Dept,Time_gap,Arr,Day_time,gap_in_second
1,H,Tue,1900-01-01 08:36:00,0 days 00:14:19,1900-01-01 08:50:19,8,859
3,H,Wed,1900-01-01 06:41:00,0 days 00:14:48,1900-01-01 06:55:48,6,888
5,H,Thu,1900-01-01 06:40:00,0 days 00:13:19,1900-01-01 06:53:19,6,799
7,H,Thu,1900-01-01 12:56:00,0 days 00:13:04,1900-01-01 13:09:04,12,784
10,H,Tue,1900-01-01 08:46:00,0 days 00:15:32,1900-01-01 09:01:32,8,932


In [36]:
# thời gian tới trường trung bình theo thời điểm bắt đầu đi trong ngày
time_to_U = df_H.groupby('Day_time').gap_in_second.mean()
# giờ trung bình bắt đầu tới trường theo ngày trong tuần
wday = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
d = {i:df_H.loc[df_H.Week_day==i, 'Arr'].mean() for i in wday}
d

{'Mon': Timestamp('1900-01-01 07:26:59.350000384'),
 'Tue': Timestamp('1900-01-01 08:30:14.166666496'),
 'Wed': Timestamp('1900-01-01 08:42:27.639999744'),
 'Thu': Timestamp('1900-01-01 08:45:59.866666496'),
 'Fri': Timestamp('1900-01-01 07:34:24.160000256'),
 'Sat': Timestamp('1900-01-01 11:00:08.714285824'),
 'Sun': Timestamp('1900-01-01 10:43:40')}

In [37]:
# số lượt di chuyển tới trường & về nhà
df.From.value_counts()

H    232
U    172
Name: From, dtype: int64